## MNIST Classifier

In [86]:
%matplotlib inline
%config inlineBackend.figure_format = "retina"

import numpy as np
import torch
#import helper

import matplotlib.pyplot as plt

from torchvision import datasets, transforms
#import torchvision.datasets as datasets

In [3]:
def activation(x):
    return (1 / (1 + torch.exp(-x)))

In [87]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [88]:
trainset = datasets.MNIST(root = './data', download = True, train = True, transform = transform)

In [183]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True)
len(trainloader)

938

In [7]:
dataiter = iter(trainloader)
images, label = dataiter.next()
print(type(images))
print(images.shape)
print(label.shape)

<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64])


In [8]:
inputs = images.view(64, -1)
print(inputs.shape)

torch.Size([64, 784])


In [9]:
torch.manual_seed(7)

W1 = torch.randn(784, 256)
B1 = torch.randn(256)

W2 = torch.randn(256, 10)
B2 = torch.randn(10)

In [10]:
h1 = activation(torch.mm(inputs, W1) + B1)

h_output = torch.mm(h1, W2) + B2

In [11]:
h_output[0]

tensor([  2.8918,  -5.7706,   6.3215,  -2.4816,  -8.7866,   9.0465,  -1.3260,
         -0.1009,   1.2504, -12.0129])

In [12]:
def softmax(x):
    x = torch.exp(x)
    output_sum = torch.sum(x, dim = 1)
    return (x / output_sum.view(output_sum.shape[0], 1))

In [13]:
output_tensor = softmax(h_output)
print(torch.sum(output_tensor, dim = 1))

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])


## Introducing the NN module (Training procedure)

In [90]:
from torch import nn, optim
import torch.nn.functional as F

<b>Negative-Log-Likelihood loss.</b>
Think of this as dot product of the hot-encoded label vector (e.g [0, 0, 0, 0, 0, 0, 1, 0, 0, 0 ] for label 7) and the output probabilty vector (e.g [0.0023, 0.00132,... 0.3477...] for label 7). This will produce a sum: 0 + 0 + 0 + 0 + 0 + 0 + 0.3477 + 0 + 0 + 0 = 0.3477. This is very low and thus knows there's a need for more training.

In [91]:
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim = 1))  #define the model

criterion = nn.NLLLoss() #define the loss function. 
optimizer = optim.SGD(model.parameters(), lr = 0.01) #define optimization function. SGD. Responsible for updating the parameters

In [96]:
for epoch in range(5):
    running_loss = 0
    for images, label in trainloader:
        images = images.view(images.shape[0], -1)
        
        optimizer.zero_grad() #empty all gradients

        logits = model.forward(images)  #Run a forward pass 
        loss = criterion(logits, label) #Calculate the loss matrix between the label and the predictions (images.shape[0], 10)
        
        loss.backward()  #Calculate and populate the gradients from the loss to all parameters: weights from input to loss
        optimizer.step() #calculate the new values based on the gradients at each weight/node.
        
        running_loss += loss.item() # .item() gets the real value of the tensor
        
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")
            

Training loss: 1.0732263112500278
Training loss: 0.392752694994656
Training loss: 0.3284683593475361
Training loss: 0.29699987728299615
Training loss: 0.2715720067273325


## Testing

In [177]:
testset = datasets.MNIST(root = './data', download = False, train = False, transform = transform)
testloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True)

In [178]:
images, label = next(iter(testloader))

In [190]:
img = images[4].view(1, 784)
print(label[4].item())

with torch.no_grad():
    logits = model.forward(img)
    
ps = F.softmax(logits, dim = 1)

2


In [191]:
logits.shape

torch.Size([1, 10])

In [175]:
print(model[0].weight)

Parameter containing:
tensor([[ 0.0159,  0.0214, -0.0183,  ...,  0.0027, -0.0114, -0.0105],
        [-0.0285, -0.0176, -0.0036,  ...,  0.0172, -0.0122, -0.0057],
        [ 0.0301, -0.0019, -0.0217,  ...,  0.0204, -0.0005,  0.0285],
        ...,
        [ 0.0166, -0.0106,  0.0016,  ..., -0.0009,  0.0190, -0.0051],
        [ 0.0323, -0.0122,  0.0114,  ..., -0.0092,  0.0064, -0.0314],
        [-0.0155,  0.0021,  0.0133,  ..., -0.0047, -0.0085, -0.0038]],
       requires_grad=True)


In [168]:
for index, ground_truth in zip(range(64), label):
    print(f"Index: {index}, Label: {ground_truth}")

Index: 0, Label: 5
Index: 1, Label: 5
Index: 2, Label: 7
Index: 3, Label: 6
Index: 4, Label: 8
Index: 5, Label: 8
Index: 6, Label: 4
Index: 7, Label: 5
Index: 8, Label: 2
Index: 9, Label: 9
Index: 10, Label: 0
Index: 11, Label: 0
Index: 12, Label: 3
Index: 13, Label: 7
Index: 14, Label: 9
Index: 15, Label: 2
Index: 16, Label: 4
Index: 17, Label: 1
Index: 18, Label: 3
Index: 19, Label: 8
Index: 20, Label: 6
Index: 21, Label: 6
Index: 22, Label: 3
Index: 23, Label: 0
Index: 24, Label: 3
Index: 25, Label: 2
Index: 26, Label: 8
Index: 27, Label: 8
Index: 28, Label: 7
Index: 29, Label: 7
Index: 30, Label: 7
Index: 31, Label: 4
Index: 32, Label: 2
Index: 33, Label: 9
Index: 34, Label: 8
Index: 35, Label: 7
Index: 36, Label: 2
Index: 37, Label: 1
Index: 38, Label: 0
Index: 39, Label: 2
Index: 40, Label: 3
Index: 41, Label: 5
Index: 42, Label: 4
Index: 43, Label: 3
Index: 44, Label: 5
Index: 45, Label: 1
Index: 46, Label: 5
Index: 47, Label: 9
Index: 48, Label: 3
Index: 49, Label: 5
Index: 50,